In [1]:
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,MinMaxScaler
import calendar
import datetime
from datetime import datetime
%matplotlib inline

/usr/local/anaconda3/lib/python3.5/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [2]:
#read the data.
car_df = pd.read_csv('../../Data/newCardata.csv')

print(car_df.shape)
car_df.head()


(15419, 33)


,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,...,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange-Claim,NumberOfCars,Year,BasePolicy
0,Dec,5,Wednesday,Honda,Urban,Tuesday,Jan,1,Female,Single,...,3 years,26 to 30,No,No,External,none,1 year,3 to 4,1994,Liability
1,Jan,3,Wednesday,Honda,Urban,Monday,Jan,4,Male,Single,...,6 years,31 to 35,Yes,No,External,none,no change,1 vehicle,1994,Collision
2,Oct,5,Friday,Honda,Urban,Thursday,Nov,2,Male,Married,...,7 years,41 to 50,No,No,External,none,no change,1 vehicle,1994,Collision
3,Jun,2,Saturday,Toyota,Rural,Friday,Jul,1,Male,Married,...,more than 7,51 to 65,Yes,No,External,more than 5,no change,1 vehicle,1994,Liability
4,Jan,5,Monday,Honda,Urban,Tuesday,Feb,2,Female,Single,...,5 years,31 to 35,No,No,External,none,no change,1 vehicle,1994,Collision


In [32]:
#replace map
replace_Month = {'Month':{'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,
                          'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12}}

replace_MonthClaimed = {'MonthClaimed':{'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,
                          'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12}}

car_df.replace(replace_Month,inplace=True)
car_df.replace(replace_MonthClaimed,inplace=True)

car_df.head()

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,...,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange-Claim,NumberOfCars,Year,BasePolicy
0,12,5,Wednesday,Honda,Urban,Tuesday,1,1,Female,Single,...,3 years,26 to 30,No,No,External,none,1 year,3 to 4,1994,Liability
1,1,3,Wednesday,Honda,Urban,Monday,1,4,Male,Single,...,6 years,31 to 35,Yes,No,External,none,no change,1 vehicle,1994,Collision
2,10,5,Friday,Honda,Urban,Thursday,11,2,Male,Married,...,7 years,41 to 50,No,No,External,none,no change,1 vehicle,1994,Collision
3,6,2,Saturday,Toyota,Rural,Friday,7,1,Male,Married,...,more than 7,51 to 65,Yes,No,External,more than 5,no change,1 vehicle,1994,Liability
4,1,5,Monday,Honda,Urban,Tuesday,2,2,Female,Single,...,5 years,31 to 35,No,No,External,none,no change,1 vehicle,1994,Collision


In [33]:
#function to calculate the no of days passed between the accident and the claims.
# Reporting Gap:

def get_day(year,month,weekOfMonth,dayOfWeek):
    count = 0
    c = calendar.TextCalendar(calendar.SUNDAY)
    l = []
    for i in c.itermonthdates(year,month):
        l.append(i)
    for j in range(len(l)):
        day = calendar.day_name[l[j].weekday()]
        if day == dayOfWeek:
            count += 1
        if count == weekOfMonth:
            return l[j]
            break

def differ_days(date1,date2):
    a = date1
    b = date2
    return (a - b).days


In [34]:

#checking the instance of the data
if(car_df['MonthClaimed'][0]-car_df['Month'][0]) < 0:
    date1= get_day(1994,12,5,'Wednesday')
    date2= get_day(1995,1,1,'Tuesday')
    print(date1,date2)

1994-12-28 1995-01-03


In [35]:
day_diff = np.zeros((car_df.shape[0],1))
i = 0
for i in range(car_df.shape[0]):
    
    if(car_df['MonthClaimed'][i]-car_df['Month'][i]) < 0:
        year2 = car_df['Year'][i] + 1
        month2 = car_df['MonthClaimed'][i]
        week2 = car_df['WeekOfMonthClaimed'][i]
        day2 = car_df['DayOfWeekClaimed'][i]
        year1 = car_df['Year'][i]
        month1 = car_df['Month'][i]
        week1 = car_df['WeekOfMonth'][i]
        day1 = car_df['DayOfWeek'][i]
        day_diff[i] = differ_days(get_day(year2,month2,week2,day2),get_day(year1,month1,week1,day1))
    else:
        year2 = car_df['Year'][i]
        month2 = car_df['MonthClaimed'][i]
        week2 = car_df['WeekOfMonthClaimed'][i]
        day2 = car_df['DayOfWeekClaimed'][i]
        year1 = car_df['Year'][i]
        month1 = car_df['Month'][i]
        week1 = car_df['WeekOfMonth'][i]
        day1 = car_df['DayOfWeek'][i]
        day_diff[i] = differ_days(get_day(year2,month2,week2,day2),get_day(year1,month1,week1,day1))
        

In [36]:
#adding column to the existing dataframe
car_df['daysDiff'] = day_diff 
car_df.head(3)


,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,...,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange-Claim,NumberOfCars,Year,BasePolicy,daysDiff
0,12,5,Wednesday,Honda,Urban,Tuesday,1,1,Female,Single,...,26 to 30,No,No,External,none,1 year,3 to 4,1994,Liability,6.0
1,1,3,Wednesday,Honda,Urban,Monday,1,4,Male,Single,...,31 to 35,Yes,No,External,none,no change,1 vehicle,1994,Collision,5.0
2,10,5,Friday,Honda,Urban,Thursday,11,2,Male,Married,...,41 to 50,No,No,External,none,no change,1 vehicle,1994,Collision,13.0


In [37]:
#now drop the original attibutes, like 'Month' column(we don't need anymore)
#now drop the original attibutes, like 'Month' column(we don't need anymore)
car_df.drop(['Month'],axis=1,inplace=True)
car_df.drop(['MonthClaimed'],axis=1,inplace=True)
car_df.drop(['DayOfWeek'],axis=1,inplace=True)
car_df.drop(['DayOfWeekClaimed'],axis=1,inplace=True)
car_df.drop(['WeekOfMonth'],inplace=True,axis=1)
car_df.drop(['WeekOfMonthClaimed'],inplace=True,axis=1)
car_df.drop(['PolicyType'],axis=1,inplace=True)
car_df.drop(['AgeOfPolicyHolder'],inplace=True,axis=1)
car_df.drop(['Year'],inplace=True,axis=1)
car_df.drop(['RepNumber'],axis=1,inplace=True)
car_df.drop(['PolicyNumber'],inplace=True,axis=1)

In [38]:
car_df.head()

,Make,AccidentArea,Sex,MaritalStatus,Age,Fault,VehicleCategory,VehiclePrice,FraudFound,Deductible,...,PastNumberOfClaims,AgeOfVehicle,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange-Claim,NumberOfCars,BasePolicy,daysDiff
0,Honda,Urban,Female,Single,21,Policy Holder,Sport,"more than 69,000",No,300,...,none,3 years,No,No,External,none,1 year,3 to 4,Liability,6.0
1,Honda,Urban,Male,Single,34,Policy Holder,Sport,"more than 69,000",No,400,...,none,6 years,Yes,No,External,none,no change,1 vehicle,Collision,5.0
2,Honda,Urban,Male,Married,47,Policy Holder,Sport,"more than 69,000",No,400,...,1,7 years,No,No,External,none,no change,1 vehicle,Collision,13.0
3,Toyota,Rural,Male,Married,65,Third Party,Sport,"20,000 to 29,000",No,400,...,1,more than 7,Yes,No,External,more than 5,no change,1 vehicle,Liability,20.0
4,Honda,Urban,Female,Single,27,Third Party,Sport,"more than 69,000",No,400,...,none,5 years,No,No,External,none,no change,1 vehicle,Collision,15.0


In [39]:
#get the lable of the datasets


label_Number = LabelEncoder()  #object of lable encoder

#conver label to Number
car_df['FraudFound'] = label_Number.fit_transform(car_df['FraudFound'].astype('str'))

yLable = car_df['FraudFound']

#drop the lable from the dataset
car_df.drop(['FraudFound'],inplace=True,axis=1)

print(yLable.shape)

(15419,)


In [40]:
car_df.head()

,Make,AccidentArea,Sex,MaritalStatus,Age,Fault,VehicleCategory,VehiclePrice,Deductible,DriverRating,...,PastNumberOfClaims,AgeOfVehicle,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange-Claim,NumberOfCars,BasePolicy,daysDiff
0,Honda,Urban,Female,Single,21,Policy Holder,Sport,"more than 69,000",300,1,...,none,3 years,No,No,External,none,1 year,3 to 4,Liability,6.0
1,Honda,Urban,Male,Single,34,Policy Holder,Sport,"more than 69,000",400,4,...,none,6 years,Yes,No,External,none,no change,1 vehicle,Collision,5.0
2,Honda,Urban,Male,Married,47,Policy Holder,Sport,"more than 69,000",400,3,...,1,7 years,No,No,External,none,no change,1 vehicle,Collision,13.0
3,Toyota,Rural,Male,Married,65,Third Party,Sport,"20,000 to 29,000",400,2,...,1,more than 7,Yes,No,External,more than 5,no change,1 vehicle,Liability,20.0
4,Honda,Urban,Female,Single,27,Third Party,Sport,"more than 69,000",400,1,...,none,5 years,No,No,External,none,no change,1 vehicle,Collision,15.0


In [41]:

#select all the attributes of type object
carObject= car_df.select_dtypes(include=['object']).copy()

#drop the attributes of type object
car_df.drop(car_df.select_dtypes(['object']),inplace=True,axis=1)

car_df.head()
#print(type(carDate))

,Age,Deductible,DriverRating,daysDiff
0,21,300,1,6.0
1,34,400,4,5.0
2,47,400,3,13.0
3,65,400,2,20.0
4,27,400,1,15.0


In [42]:
car_df.replace(0,17) #make all zeros to 17

,Age,Deductible,DriverRating,daysDiff
0,21,300,1,6.0
1,34,400,4,5.0
2,47,400,3,13.0
3,65,400,2,20.0
4,27,400,1,15.0
5,20,400,3,12.0
6,36,400,1,9.0
7,17,400,4,137.0
8,30,400,4,4.0
9,42,400,1,1.0


In [43]:
#normalization of feature to bring the value in the range [0,1]

minMaxScale= MinMaxScaler() #minMax scaler
car_df= minMaxScale.fit_transform(car_df)

#converting numpyarry to dataframe
car_df = pd.DataFrame(car_df)
car_df.head()




/usr/local/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,0,1,2,3
0,0.2625,0.00,0.000000,0.071795
1,0.4250,0.25,1.000000,0.069231
2,0.5875,0.25,0.666667,0.089744
3,0.8125,0.25,0.333333,0.107692
4,0.3375,0.25,0.000000,0.094872


In [44]:
#one hot encoding
car_copy = carObject.copy()
car_copy = pd.get_dummies(car_copy,columns=['Make'],prefix=['Make'])
car_copy = pd.get_dummies(car_copy,columns=['AccidentArea'],prefix=['AccidentArea'])
car_copy = pd.get_dummies(car_copy,columns=['Sex'],prefix=['Sex'])
car_copy = pd.get_dummies(car_copy,columns=['MaritalStatus'],prefix=['MartalStatus'])
car_copy = pd.get_dummies(car_copy,columns=['Fault'],prefix=['Fault'])
car_copy = pd.get_dummies(car_copy,columns=['VehicleCategory'],prefix=['VehicleCategory'])
car_copy = pd.get_dummies(car_copy,columns=['VehiclePrice'],prefix=['VehiclePrice'])
car_copy = pd.get_dummies(car_copy,columns=['Days:Policy-Accident'],prefix=['Days:Policy-Accident'])
car_copy = pd.get_dummies(car_copy,columns=['Days:Policy-Claim'],prefix=['Days:Policy-Claim'])
car_copy = pd.get_dummies(car_copy,columns=['PastNumberOfClaims'],prefix=['PastNumberOfClaims'])
car_copy = pd.get_dummies(car_copy,columns=['AgeOfVehicle'],prefix=['AgeOfVehicle'])
car_copy = pd.get_dummies(car_copy,columns=['PoliceReportFiled'],prefix=['PoliceReportFiled'])
car_copy = pd.get_dummies(car_copy,columns=['WitnessPresent'],prefix=['WitnessPresent'])
car_copy = pd.get_dummies(car_copy,columns=['AgentType'],prefix=['AgentType'])
car_copy = pd.get_dummies(car_copy,columns=['NumberOfSuppliments'],prefix=['NumberOfSuppliments'])
car_copy = pd.get_dummies(car_copy,columns=['AddressChange-Claim'],prefix=['AddressChange-Claim'])
car_copy = pd.get_dummies(car_copy,columns=['NumberOfCars'],prefix=['NumberOfCars'])
car_copy = pd.get_dummies(car_copy,columns=['BasePolicy'],prefix=['BasePolicy'])




car_copy.head(3)

,Make_Accura,Make_BMW,Make_Chevrolet,Make_Dodge,Make_Ferrari,Make_Ford,Make_Honda,Make_Jaguar,Make_Lexus,Make_Mazda,...,AddressChange-Claim_no change,AddressChange-Claim_under 6 months,NumberOfCars_1 vehicle,NumberOfCars_2 vehicles,NumberOfCars_3 to 4,NumberOfCars_5 to 8,NumberOfCars_more than 8,BasePolicy_All Perils,BasePolicy_Collision,BasePolicy_Liability
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [45]:
car_copy['Age']= car_df[0]
car_copy['Deductible']= car_df[2]
car_copy['DriverRating']= car_df[2]
car_copy['DaysDiff']= car_df[3]
car_copy['Lable']= yLable

car_copy.head(3)




,Make_Accura,Make_BMW,Make_Chevrolet,Make_Dodge,Make_Ferrari,Make_Ford,Make_Honda,Make_Jaguar,Make_Lexus,Make_Mazda,...,NumberOfCars_5 to 8,NumberOfCars_more than 8,BasePolicy_All Perils,BasePolicy_Collision,BasePolicy_Liability,Age,Deductible,DriverRating,DaysDiff,Lable
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.2625,0.000000,0.000000,0.071795,0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.4250,1.000000,1.000000,0.069231,0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.5875,0.666667,0.666667,0.089744,0


In [47]:
#saving the final data after preprocessing:

#saving dataframe to the csv format
car_copy.to_csv('oneHotPreprocessed.csv',sep=' ',encoding='utf-8',index=None)


In [52]:
new= pd.read_csv('./../../Data/oneHotPreprocess.csv')
new.head()

,Make_Accura,Make_BMW,Make_Chevrolet,Make_Dodge,Make_Ferrari,Make_Ford,Make_Honda,Make_Jaguar,Make_Lexus,Make_Mazda,...,NumberOfCars_5 to 8,NumberOfCars_more than 8,BasePolicy_All Perils,BasePolicy_Collision,BasePolicy_Liability,Age,Deductible,DriverRating,DaysDiff,Lable
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0.2625,0.000000,0.000000,0.071795,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0.4250,1.000000,1.000000,0.069231,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0.5875,0.666667,0.666667,0.089744,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0.8125,0.333333,0.333333,0.107692,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0.3375,0.000000,0.000000,0.094872,0
